In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm

In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Sampler, TensorDataset

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
model.to('cuda:0')

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [6]:
df = pd.read_csv('/kaggle/input/nlpdlfr/films_prepared_v3.csv')

In [7]:
df.drop('characters.1', inplace=True, axis=1)

In [8]:
df.replics = df.replics.apply(lambda x: eval(x))
df.chrono = df.chrono.apply(lambda x: eval(x))
df.characters = df.characters.apply(lambda x: eval(x))
df.genders = df.genders.apply(lambda x: eval(x))
df.genders = df.genders.apply(lambda x: x if '?' not in x else np.nan)

df['replics_len'] = df.replics.apply(lambda x: len(x))

df_new = df[df.replics_len < 5]
df_new.dropna(inplace=True)
df_new = df_new.sample(200, random_state=18) # Выберем 200 диалогов небольшой длины
df_new.reset_index(inplace=True, drop=True)

/tmp/ipykernel_28/3879652195.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.dropna(inplace=True)


In [9]:
df_new

,movieID,chrono,characters,genders,replics,movie_title,genres,replics_len
0,m615,"[L665758, L665759, L665760, L665761]","[u9017, u9019, u9017, u9019]","[M, m, M, m]",[The ancient masters promised impossibilities ...,young frankenstein,"['comedy', 'sci-fi']",4
1,m188,"[L522640, L522641]","[u2852, u2860]","[f, m]","[Tommy...!, I'm all right.]",someone to watch over me,"['action', 'crime', 'drama', 'romance', 'thril...",2
2,m401,"[L309132, L309133]","[u6033, u6035]","[m, f]",[I don't think so Thea. He has spoken of your ...,isle of the dead,"['drama', 'horror', 'mystery', 'thriller']",2
3,m570,"[L609752, L609753, L609754, L609755]","[u8410, u8409, u8410, u8409]","[f, M, f, M]",[We're American soldiers. We don't abandon pe...,three kings,"['action', 'adventure', 'comedy', 'drama', 'war']",4
4,m235,"[L15348, L15349]","[u3572, u3574]","[m, m]",[Eh. I thought you were looking after the old ...,a hard day's night,"['comedy', 'music']",2
...,...,...,...,...,...,...,...,...
195,m403,"[L320137, L320138, L320139]","[u6068, u6074, u6068]","[m, f, m]","[Sean's awake., Door's locked., Good.]",jaws 2,['thriller'],3
196,m218,"[L607479, L607480]","[u3299, u3293]","[m, m]","[Let's get out of here., Cheer up you've neutr...",thirteen days,"['drama', 'history', 'thriller']",2
197,m427,"[L359662, L359663]","[u6413, u6410]","[m, f]","[It's okay little baby., Quit.]",love & basketball,"['drama', 'romance', 'sport']",2
198,m90,"[L284854, L284855]","[u1357, u1364]","[m, f]","[When did you deliver this first?, Who did you...",his girl friday,"['comedy', 'drama', 'romance']",2


In [10]:
def convert_to_dataset_torch(data: str):
    input_ids = []
    attention_masks = []
    
    encoded_dict = tokenizer.encode_plus(data, 
                                             max_length=130,
                                             pad_to_max_length=True,
                                             return_attention_mask=True, 
                                             return_tensors='pt', 
                                             truncation=True)

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])
        
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    input_ids.to(dtype=torch.long)
    attention_masks.to(dtype=torch.long)
    
    return input_ids, attention_masks

In [11]:
labels = [
    'Positive',
    'Negative',
    'Neutral'
]
mapping = {i: k for i, k in enumerate(labels)}

In [10]:
# Разметка данных

preds_all = []
for i in tqdm(df_new.replics):
    preds_dial = []
    for k in i:
        converted = convert_to_dataset_torch(k)
        preds = model(input_ids=converted[0].to('cuda:0'), 
                      attention_mask=converted[1].to('cuda:0')).logits
        predictions = torch.argmax(torch.nn.Softmax()(preds), axis=1)
        preds_dial.append(predictions.item())
    preds_all.append(preds_dial)
        
      

  0%|          | 0/200 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/tmp/ipykernel_29/2488892409.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.argmax(torch.nn.Softmax()(preds), axis=1)
100%|██████████| 200/200 [00:07<00:00, 25.85it/s]


In [11]:
df_new['predictions'] = pd.Series(preds_all)

In [12]:
df_new

,movieID,chrono,characters,genders,replics,movie_title,genres,replics_len,predictions
0,m615,"[L665758, L665759, L665760, L665761]","[u9017, u9019, u9017, u9019]","[M, m, M, m]",[The ancient masters promised impossibilities ...,young frankenstein,"['comedy', 'sci-fi']",4,"[1, 1, 2, 2]"
1,m188,"[L522640, L522641]","[u2852, u2860]","[f, m]","[Tommy...!, I'm all right.]",someone to watch over me,"['action', 'crime', 'drama', 'romance', 'thril...",2,"[1, 2]"
2,m401,"[L309132, L309133]","[u6033, u6035]","[m, f]",[I don't think so Thea. He has spoken of your ...,isle of the dead,"['drama', 'horror', 'mystery', 'thriller']",2,"[1, 0]"
3,m570,"[L609752, L609753, L609754, L609755]","[u8410, u8409, u8410, u8409]","[f, M, f, M]",[We're American soldiers. We don't abandon pe...,three kings,"['action', 'adventure', 'comedy', 'drama', 'war']",4,"[2, 1, 0, 1]"
4,m235,"[L15348, L15349]","[u3572, u3574]","[m, m]",[Eh. I thought you were looking after the old ...,a hard day's night,"['comedy', 'music']",2,"[0, 1]"
...,...,...,...,...,...,...,...,...,...
195,m403,"[L320137, L320138, L320139]","[u6068, u6074, u6068]","[m, f, m]","[Sean's awake., Door's locked., Good.]",jaws 2,['thriller'],3,"[1, 0, 2]"
196,m218,"[L607479, L607480]","[u3299, u3293]","[m, m]","[Let's get out of here., Cheer up you've neutr...",thirteen days,"['drama', 'history', 'thriller']",2,"[1, 2]"
197,m427,"[L359662, L359663]","[u6413, u6410]","[m, f]","[It's okay little baby., Quit.]",love & basketball,"['drama', 'romance', 'sport']",2,"[2, 0]"
198,m90,"[L284854, L284855]","[u1357, u1364]","[m, f]","[When did you deliver this first?, Who did you...",his girl friday,"['comedy', 'drama', 'romance']",2,"[1, 1]"


In [13]:
df_new.to_csv('labeled_v2.csv')